In [ ]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

import matplotlib.pyplot as plt

from datetime import datetime

In [ ]:
# Load and examine the dataset
df = pd.read_csv('/Users/rotation/Desktop/Dataclass/capstone-project/archive_boston/crime.csv', encoding='latin-1')
df.head()

In [ ]:
# Examine available columns
df.columns.tolist()

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
#  Import and read the crime_data.csv.
df = pd.read_csv('/Users/rotation/Desktop/Dataclass/capstone-project/archive_boston/crime.csv', encoding='latin-1')
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [3]:
# Drop the non-beneficial ID columns, 'HOUR' and 'UCR_PART'.
#  YOUR CODE GOES HERE
df= df.drop(['HOUR', 'UCR_PART'],1)
df.head()

/Users/rotation/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [4]:
# Determine the number of unique values in each column.

df.nunique()

INCIDENT_NUMBER        282517
OFFENSE_CODE              222
OFFENSE_CODE_GROUP         67
OFFENSE_DESCRIPTION       244
DISTRICT                   12
REPORTING_AREA            879
SHOOTING                    1
OCCURRED_ON_DATE       233229
YEAR                        4
MONTH                      12
DAY_OF_WEEK                 7
STREET                   4657
Lat                     18178
Long                    18178
Location                18194
dtype: int64

In [ ]:
df.apply(pd.Series.nunique)


In [ ]:
df.drop_duplicates(subset="INCIDENT_NUMBER", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_DESCRIPTION", inplace=True)

In [ ]:
df.describe().T

In [ ]:
# Get transpose of DataFrame
transposedDf = df.T
print(transposedDf)

In [ ]:
#df.groupby(['INCIDENT_NUMBER','DISTRICT']).count()


In [ ]:
# FREQUENCY
print(df.OFFENSE_CODE_GROUP.value_counts())


In [ ]:
# create a dict of lists
data = {'OFFENSE_CODE_GROUP': ['Larceny',
 'Vandalism',
 'Towed',
 'Investigate Property',
 'Motor Vehicle Accident Response',
 'Auto Theft',
 'Verbal Disputes',
 'Robbery',
 'Fire Related Reports',
 'Other',
 'Property Lost',
 'Medical Assistance',
 'Assembly or Gathering Violations',
 'Larceny From Motor Vehicle',
 'Residential Burglary',
 'Simple Assault',
 'Violations',
 'Harassment',
 'Ballistics',
 'Property Found',
 'Police Service Incidents',
 'Drug Violation',
 'Missing Person Reported',
 'Investigate Person',
 'Fraud',
 'Aggravated Assault',
 'License Plate Related Incidents',
 'Other Burglary',
 'Warrant Arrests',
 'Disorderly Conduct',
 'Harbor Related Incidents',
 'Counterfeiting',
 'Liquor Violation',
 'Firearm Discovery',
 'Landlord/Tenant Disputes',
 'Auto Theft Recovery',
 'Service',
 'Operating Under the Influence',
 'Confidence Games',
 'Restraining Order Violations',
 'Firearm Violations',
 'Missing Person Located',
 'License Violation',
 'Commercial Burglary',
 'Search Warrants',
 'Recovered Stolen Property',
 'Offenses Against Child / Family',
 'Prostitution',
 'Bomb Hoax',
 'Evading Fare',
 'Property Related Damage',
 'Prisoner Related Incidents',
 'Homicide',
 'Embezzlement',
 'Explosives',
 'Arson',
 'Criminal Harassment',
 'HOME INVASION',
 'Phone Call Complaints',
 'Aircraft',
 'Biological Threat',
 'Manslaughter',
 'Gambling',
 'HUMAN TRAFFICKING',
 'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
 'Burglary - No Property Taken'],
       'OFFENSE_CODE':[619,
 1402,
 3410,
 3114,
 3820,
 724,
 3301,
 301,
 3108,
 2647,
 3201,
 3006,
 3305,
 614,
 522,
 802,
 2907,
 2629,
 2662,
 3207,
 3109,
 1843,
 3501,
 3115,
 1102,
 423,
 3205,
 561,
 3125,
 2405,
 3116,
 1001,
 2646,
 3119,
 3112,
 735,
 3402,
 2101,
 1106,
 2007,
 1503,
 3502,
 3111,
 540,
 3130,
 1304,
 2005,
 1605,
 2648,
 2632,
 3106,
 2619,
 111,
 1201,
 3123,
 900,
 2670,
 2010,
 2628,
 3122,
 2672,
 121,
 1901,
 1610,
 1620,
 527]}
        
        
# Creating a dataframe from the dict
df = pd.DataFrame(data)

df.head()

In [ ]:
from scipy.stats import mode

def unique_nan(s):
    return s.nunique(dropna=False)
def count_nulls(s):
    return s.size - s.count()

agg_func_custom_count = {
    'OFFENSE_CODE_GROUP': ['count', 'nunique', 'size', unique_nan, count_nulls, set]
}
df.groupby(['DAY_OF_WEEK']).agg(agg_func_custom_count)

In [ ]:
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.OFFENSE_CODE.tolist()

In [ ]:
df.drop_duplicates(subset="OFFENSE_CODE_GROUP", inplace=True)
df.OFFENSE_CODE_GROUP.tolist()

In [ ]:
df.OFFENSE_CODE_GROUP

In [ ]:
data = {'Larceny': 619, 'Vandalism': 1402, 'Towed': 3410, 'Investigate Property': 3114}
names = list(data.keys())
values = list(data.values())

fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=True)
axs[1].scatter(names, values)
axs[0].bar(names, values)
fig.suptitle('Boston Crimes by Type')

In [ ]:
data = {'Motor Vehicle Accident Response': 3820, 'Vandalism': 1402, 'Towed': 3410, 'Investigate Property': 3114}
names = list(data.keys())
values = list(data.values())

fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=True)
axs[0].bar(names, values)
axs[1].scatter(names, values)
fig.suptitle('Boston Crimes by Type')

In [ ]:
data = [[ 619,
 1402,
 3410,
 3114,
 3820,
 724,
 3301,
 301,
 3108,
 2647,
 3201,
 3006,
 3305,
 614,
 522,
 802,
 2907,
 2629,
 2662,
 3207,
 3109,
 1843,
 3501,
 3115,
 1102,
 423,
 3205,
 561,
 3125,
 2405,
 3116,
 1001,
 2646,
 3119,
 3112,
 735,
 3402,
 2101,
 1106,
 2007,
 1503,
 3502,
 3111,
 540,
 3130,
 1304,
 2005,
 1605,
 2648,
 2632,
 3106,
 2619,
 111,
 1201,
 3123,
 900,
 2670,
 2010,
 2628,
 3122,
 2672,
 121,
 1901,
 1610,
 1620,
 527]]

columns = ('Larceny',
 'Vandalism',
 'Towed',
 'Investigate Property',
 'Motor Vehicle Accident Response',
 'Auto Theft',
 'Verbal Disputes',
 'Robbery',
 'Fire Related Reports',
 'Other',
 'Property Lost',
 'Medical Assistance',
 'Assembly or Gathering Violations',
 'Larceny From Motor Vehicle',
 'Residential Burglary',
 'Simple Assault',
 'Violations',
 'Harassment',
 'Ballistics',
 'Property Found',
 'Police Service Incidents',
 'Drug Violation',
 'Missing Person Reported',
 'Investigate Person',
 'Fraud',
 'Aggravated Assault',
 'License Plate Related Incidents',
 'Other Burglary',
 'Warrant Arrests',
 'Disorderly Conduct',
 'Harbor Related Incidents',
 'Counterfeiting',
 'Liquor Violation',
 'Firearm Discovery',
 'Landlord/Tenant Disputes',
 'Auto Theft Recovery',
 'Service',
 'Operating Under the Influence',
 'Confidence Games',
 'Restraining Order Violations',
 'Firearm Violations',
 'Missing Person Located',
 'License Violation',
 'Commercial Burglary',
 'Search Warrants',
 'Recovered Stolen Property',
 'Offenses Against Child / Family',
 'Prostitution',
 'Bomb Hoax',
 'Evading Fare',
 'Property Related Damage',
 'Prisoner Related Incidents',
 'Homicide',
 'Embezzlement',
 'Explosives',
 'Arson',
 'Criminal Harassment',
 'HOME INVASION',
 'Phone Call Complaints',
 'Aircraft',
 'Biological Threat',
 'Manslaughter',
 'Gambling',
 'HUMAN TRAFFICKING',
 'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
 'Burglary - No Property Taken')
rows = ['%d year' % x for x in (100, 50, 20, 10, 5)]

values = np.arange(0, 2500, 500)
value_increment = 1000

# Get some pastel shades for the colors
colors = plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
n_rows = len(data)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.002, bottom=0.002)

plt.ylabel("Loss in ${0}'s".format(value_increment))
plt.yticks(values * value_increment, ['%d' % val for val in values])
plt.xticks([])
plt.title('Loss by Disaster')

plt.show()

In [ ]:
matplotlib.pyplot.table(cellText=None, cellColours=None, cellLoc='right', colWidths=None, rowLabels=None, rowColours=None, rowLoc='left', colLabels=None, colColours=None, colLoc='center', loc='bottom', bbox=None, edges='closed', **kwargs)